In [68]:
# Import libraries necesarry for web scraping with selenium
#import selenium
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service as ChromeService
#import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

import requests
from bs4 import BeautifulSoup
import json
import PyPDF2
import io
import re
import pandas as pd
# Import necessary libraries


Here are the Websites I am thinking of using:

https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/docApi

https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/topic-pages/tables/table-5

https://www.icpsr.umich.edu/web/NACJD/studies/38604/summary

https://bjs.ojp.gov/library/publications/criminal-victimization-2022

https://endrapeoncampus.org/map-and-tool/

https://ope.ed.gov/campussafety/#/compare/confirm

https://www.prri.org/research/2020-census-of-american-religion/

https://www.statista.com/statistics/221454/share-of-religious-americans-by-state/

BIG 12 (for 2024 group) FIRE & SAFETY REPORTS:

BYU https://police.byu.edu/2023-asr-provo  (page 9)

Baylor https://www.baylor.edu/risk/doc.php/399779.pdf (page 12)

UCF https://police.ucf.edu/sites/default/files/2023%20Annual%20Security%20%26%20Fire%20Safety%20Report.pdf (page 78)

Cinncinati https://www.uc.edu/content/dam/refresh/publicsafety-62/docs/ucpd/2023-annual-security-report.pdf (page 34)

Houston http://m2s-conf.uh.edu/police/records-reports/annualreport/2023-uh-annual-security-and-fire-safety-report.pdf (page 39)

Iowa https://www.police.iastate.edu/wp-content/uploads/2023/09/2022-Report-Final.pdf (page 6)

KU https://civilrights.ku.edu/sites/civilrights/files/documents/Reports/University%20of%20Kansas%20Annual%20Security%20and%20Fire%20Safety%20Report-2023-KLETC.pdf (page 60)

K-state https://www.k-state.edu/report/clery/reports/KSUCleryReport2023.pdf (page 13)

OSU https://safety.okstate.edu/police/documents/annual-security-reports/2023-annual-safety-report.pdf (page 37)

TCU https://police.tcu.edu/wp-content/uploads/2023/09/TCU-Annual-Security-Report-Fire-Safety-Report_2023.v1.pdf (page 80)

Texas Tech https://www.depts.ttu.edu/clery/reports/2023LubbockASRfinal.pdf (page 107)

West Virginia https://police.wvu.edu/files/d/a1a0a6fe-444e-4353-8e92-7dcbeb00cd27/2023-annual-security-and-fire-safety-report-with-appendix.pdf (page 36)

In [2]:
import camelot

In [115]:
BYU_table = camelot.read_pdf('BYU_report.pdf', pages='9', flavor='stream')
Baylor_tables = camelot.read_pdf('Baylor_report.pdf', pages='12,13', flavor='stream')
Cinn_table = camelot.read_pdf('Cinn_report.pdf', pages='34', flavor='stream')
Iowa_table = camelot.read_pdf('Iowa_report.pdf', pages='6', flavor='stream')
Kstate_table = camelot.read_pdf('Kstate_report.pdf', pages='13', flavor='stream')
KU_table = camelot.read_pdf('KU_report.pdf', pages='60', flavor='stream')
OSU_table = camelot.read_pdf('OSU_report.pdf', pages='37', flavor='stream')
TCU_table = camelot.read_pdf('TCU_report.pdf', pages='80', flavor='stream')
Ttech_table = camelot.read_pdf('Ttech_report.pdf', pages='107', flavor='stream')
UCF_tables = camelot.read_pdf('UCF_report.pdf', pages='78,79,80', flavor='stream')
UH_table = camelot.read_pdf('UH_report.pdf', pages='39', flavor='stream')
WV_table = camelot.read_pdf('WV_report.pdf', pages='36', flavor='stream')

In [150]:
BYU_table[0].df
# take only rows 0,1, and 9-12
BYU_sexOff = BYU_table[0].df.iloc[[1,9,10,11,12]]
# take out columns 2,4,6
BYU_sexOff = BYU_sexOff.drop([2,4,6], axis=1)
# reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
# make the first row the column names
BYU_sexOff.columns = BYU_sexOff.iloc[0]
# drop the first row
BYU_sexOff = BYU_sexOff.drop([0])
# reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
# Add the school name
BYU_sexOff['School'] = 'BYU'
# Adding the year counts together
BYU_sexOff['2020 Total'] = BYU_sexOff['2020'].astype(int).sum(axis = 1)
BYU_sexOff['2021 Total'] = BYU_sexOff['2021'].astype(int).sum(axis = 1)
BYU_sexOff['2022 Total'] = BYU_sexOff['2022'].astype(int).sum(axis = 1)
# Keep only the columns we want
BYU_sexOff = BYU_sexOff[['Year Reported','School','2020 Total','2021 Total','2022 Total']]
# Rename Year Reported to Offense
BYU_sexOff = BYU_sexOff.rename(columns={'Year Reported': 'Offense'})

BYU_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,BYU,4,7,6
1,Fondling,BYU,10,13,17
2,Incest,BYU,0,0,0
3,Statutory Rape,BYU,0,0,0


In [143]:
Baylor_tables[0].df
# Only take row 16 and below
Baylor_sexOff = Baylor_tables[0].df[16:]
Baylor_sexOff = Baylor_sexOff.drop([19], axis=0)
# reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)
# Remove the 0 and 2 columns
Baylor_sexOff = Baylor_sexOff.drop([0,2], axis=1)
# Add a row with custom values
data = {1:'Statutory Rape', 3:'0',4:'0',5:'0',6:'0',7:'0',8:'0'}
# Append the row to the dataframe
Baylor_sexOff = Baylor_sexOff.append(data, ignore_index=True)
# only keep columns 1,6
Baylor_sexOff = Baylor_sexOff[[1,6]] 
# Rename the columns
Baylor_sexOff = Baylor_sexOff.rename(columns={1: 'Offense', 6: '2020 Total'}) 
# remove the first row
Baylor_sexOff = Baylor_sexOff.drop([0], axis=0) 

Baylor_sexOff['School'] = 'Baylor'
Baylor_sexOff['2021 Total'] = ['20','10','0','0']
Baylor_sexOff['2022 Total'] = ['22','8','0','0']

# Reorder the columns
Baylor_sexOff = Baylor_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)
Baylor_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\2115857788.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Baylor_sexOff = Baylor_sexOff.append(data, ignore_index=True)


,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,Baylor,13,20,22
1,Fondling,Baylor,13,10,8
2,Incest,Baylor,0,0,0
3,Statutory Rape,Baylor,0,0,0


In [171]:
Cinn_table[0].df
# Keep rows 3, 6-9
Cinn_sexOff = Cinn_table[0].df.iloc[[3,6,7,8,9]]
# Reset the index
Cinn_sexOff = Cinn_sexOff.reset_index(drop=True)
# Make first row the columns
Cinn_sexOff.columns = Cinn_sexOff.iloc[0]
# Drop the first row
Cinn_sexOff = Cinn_sexOff.drop([0])
# Reset the index
Cinn_sexOff = Cinn_sexOff.reset_index(drop=True)
# Remove all the * from row 1
Cinn_sexOff.iloc[0,] = Cinn_sexOff.iloc[0,].str.replace('*','')
# Add the school name
Cinn_sexOff['School'] = 'CU'
# Sum the columns
Cinn_sexOff['2020 Total'] = Cinn_sexOff['2020'].astype(int).sum(axis = 1)
Cinn_sexOff['2021 Total'] = Cinn_sexOff['2021'].astype(int).sum(axis = 1)
Cinn_sexOff['2022 Total'] = Cinn_sexOff['2022'].astype(int).sum(axis = 1)
# Keep only the columns we want
Cinn_sexOff = Cinn_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Swap rows 3 and 4
Cinn_sexOff.iloc[[2,3]] = Cinn_sexOff.iloc[[3,2]]
Cinn_sexOff


C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\845297068.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Cinn_sexOff.iloc[0,] = Cinn_sexOff.iloc[0,].str.replace('*','')


,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,CU,20,34,52
1,Fondling,CU,7,20,19
2,Incest,CU,0,0,0
3,Statutory Rape,CU,0,0,0


In [176]:
Iowa_table[0].df
# Keep idex rows 0,1, 8-11
Iowa_sexOff = Iowa_table[0].df.iloc[[0,1,8,9,10,11]]
# Reset the index
Iowa_sexOff = Iowa_sexOff.reset_index(drop=True)
# Keep columns 0,16,17,18
Iowa_sexOff = Iowa_sexOff[[0,16,17,18]]
# Drop row 0 and 1
Iowa_sexOff = Iowa_sexOff.drop([0,1], axis=0)
# Reset the index
Iowa_sexOff = Iowa_sexOff.reset_index(drop=True)
# Rename the columns
Iowa_sexOff = Iowa_sexOff.rename(columns={0: 'Offense', 16: '2020 Total', 17: '2021 Total', 18: '2022 Total'})
# Add the school name
Iowa_sexOff['School'] = 'Iowa'
# Reorder the columns
Iowa_sexOff = Iowa_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]

Iowa_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,Iowa,10,22,18
1,Fondling,Iowa,4,7,6
2,Incest,Iowa,0,0,0
3,Statutory Rape,Iowa,0,0,0
